# Translating (IOS)

As explained in the previous tutorial, a ntc_rosetta driver can both parse and translate between native and yang-based models. In this tutorial we are going to translate data that complies to the openconfig model into native IOS configuration.

Let's start by loading the needed driver:

In [9]:
from ntc_rosetta import get_driver

ios = get_driver("ios", "openconfig")
ios_processor = ios()

Now we need some data:

In [18]:
system = {
    "openconfig-system:system": {
      "config": {
        "hostname": "csr1000v-1"
      },
      "clock": {
        "config": {
          "timezone-name": "CST -6 0"
        }
      },
      "dns": {
        "servers": {
          "server": [
            {
              "address": "8.8.8.8",
              "config": {
                "address": "8.8.8.8",
                "port": 53
              }
            },
            {
              "address": "8.8.4.4",
              "config": {
                "address": "8.8.4.4",
                "port": 53
              }
            },
            {
              "address": "2620:119:35::35",
              "config": {
                "address": "2620:119:35::35",
                "port": 53
              }
            }
          ]
        }
      },
      "ntp": {
        "config": {
          "enabled": True,
          "enable-ntp-auth": False
        },
        "servers": {
          "server": [
            {
              "address": "2610:20:6F96:96::6",
              "config": {
                "address": "2610:20:6F96:96::6"
              }
            }
          ]
        }
      },
      "ssh-server": {
        "config": {
          "enable": True,
          "protocol-version": "V2",
          "timeout": 60
        }
      },
      "telnet-server": {
        "config": {
          "enable": True
        }
      },
      "aaa": {
        "authentication": {
          "users": {
            "user": [
              {
                "username": "developer",
                "config": {
                  "username": "developer",
                  "password-hashed": "$1$pdQG$blOvq4Ey0SJCv3n001vnj.",
                  "role": "15"
                }
              }
            ]
          }
        }
      }
    }
}
interfaces = {
    "openconfig-interfaces:interfaces": {
        "interface": [
            {
                "name": "FastEthernet1",
                "config": {
                    "name": "FastEthernet1",
                    "type": "iana-if-type:ethernetCsmacd",
                    "loopback-mode": False,
                    "description": "This is Fa1",
                    "enabled": False
                },
                "subinterfaces": {
                    "subinterface": [
                        {
                            "index": 1,
                            "config": {
                                "index": 1,
                                "description": "This is Fa1.1"
                            }
                        },
                        {
                            "index": 2,
                            "config": {
                                "index": 2,
                                "description": "This is Fa1.2"
                            }
                        }
                    ]
                }
            },
            {
                "name": "FastEthernet3",
                "config": {
                    "name": "FastEthernet3",
                    "type": "iana-if-type:ethernetCsmacd",
                    "loopback-mode": False,
                    "description": "This is Fa3",
                    "enabled": True
                },
                "openconfig-if-ethernet:ethernet": {
                    "openconfig-vlan:switched-vlan": {
                        "config": {
                            "interface-mode": "ACCESS",
                            "access-vlan": 10
                        }
                    }
                }
            },
            {
                "name": "FastEthernet4",
                "config": {
                    "name": "FastEthernet4",
                    "type": "iana-if-type:ethernetCsmacd",
                    "loopback-mode": True,
                    "enabled": False
                },
                "openconfig-if-ethernet:ethernet": {
                    "openconfig-vlan:switched-vlan": {
                        "config": {
                            "interface-mode": "TRUNK",
                            "trunk-vlans": [
                                10,
                                20
                            ]
                        }
                    }
                }
            }
        ]
    }
}
nw_instance = {
    "openconfig-network-instance:network-instances": {
        "network-instance": [
            {
                "name": "default",
                "config": {
                    "name": "default"
                },
                "vlans": {
                    "vlan": [
                        {
                            "vlan-id": 10,
                            "config": {
                                "vlan-id": 10,
                                "name": "prod",
                                "status": "ACTIVE"
                            }
                        },
                        {
                            "vlan-id": 20,
                            "config": {
                                "vlan-id": 20,
                                "name": "dev",
                                "status": "SUSPENDED"
                            }
                        }
                    ]
                }
            }
        ]
    }
}

There is one "gotcha" to translating.  The backing library, Yangson, doesn't consider key order when building the data model.  To ensure order of operations, it is best to be deterministic.

Once we have the data, translating it to native is very simple:

In [23]:
native = str.join("\n", [ios_processor.translate(candidate=data) for data in [system, interfaces, nw_instance]])

We can verify the result by just printing it:

In [22]:
print(native)

hostname csr1000v-1
clock timezone CST -6 0
ip name-server 8.8.8.8 8.8.4.4 2620:119:35::35
ntp server 2610:20:6F96:96::6
ip ssh version 2
ip ssh time-out 60
username developer privilege 15 secret 5 $1$pdQG$blOvq4Ey0SJCv3n001vnj.

interface FastEthernet1
   no loopback mac
   description This is Fa1
   shutdown
   exit
!
interface FastEthernet1.1
   description This is Fa1.1
   exit
!
interface FastEthernet1.2
   description This is Fa1.2
   exit
!
interface FastEthernet3
   no loopback mac
   description This is Fa3
   no shutdown
   switchport mode access
   switchport access vlan 10
   exit
!
interface FastEthernet4
   loopback mac
   shutdown
   switchport mode trunk
   switchport trunk allowed vlan 10,20
   exit
!

vlan 10
   name prod
   no shutdown
   exit
!
vlan 20
   name dev
   shutdown
   exit
!

